### Name : Bilal Muhammad Khan
### Gmail : drbilal216@gmail.com
### My Competition Id : EVENT-224445

## Importing Libraries

In [1]:
import os
import shutil
import tensorflow as tf
import PIL.Image
import matplotlib.pyplot as plt

## Downloading Pretrained model

In [29]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

58889256/58889256 [==============================] - 638s 11us/step


## Building Model

In [30]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 8192)              0         
                                                                 
 dense_6 (Dense)             (None, 256)               2097408   
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16812353 (64.13 MB)
Trainable params: 16812353 (64.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## freezing the convolutional base
In Keras, you freeze a network by setting its trainable attribute to False:

In [31]:
print('This is the number of trainable weights ''before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 30


In [32]:
conv_base.trainable = False

In [33]:
 print('This is the number of trainable weights ''after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 4


## Training the model end to end with a frozen convolutional base

In [35]:
# Path for train, validation and test datasets
train_dir = 'train'
test_dir = 'test'
validation_dir = 'val'

In [36]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

In [37]:
train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest')

In [38]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [39]:
train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size=(150, 150),
                    batch_size=20,
                    class_mode='binary')

Found 5236 images belonging to 2 classes.


In [40]:
validation_generator = test_datagen.flow_from_directory(
                        validation_dir,
                        target_size=(150, 150),
                        batch_size=20,
                        class_mode='binary')

Found 16 images belonging to 2 classes.


In [41]:
model.compile(loss='binary_crossentropy',
                optimizer=optimizers.RMSprop(lr=2e-5),
                metrics=['acc'])

In [43]:
history = model.fit_generator(
                train_generator,
                steps_per_epoch=100,
                epochs=3,#30
                validation_data=validation_generator,
                validation_steps=50)

C:\Users\ESHOP\AppData\Local\Temp\ipykernel_6860\3578974842.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/3
100/100 [==============================] - ETA: 0s - loss: 0.3033 - acc: 0.8677WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 50 batches). You may need to use the repeat() function when building your dataset.


100/100 [==============================] - 260s 3s/step - loss: 0.3033 - acc: 0.8677 - val_loss: 0.6182 - val_acc: 0.7500
Epoch 2/3
100/100 [==============================] - 252s 3s/step - loss: 0.2362 - acc: 0.8970
Epoch 3/3
100/100 [==============================] - 264s 3s/step - loss: 0.2215 - acc: 0.9063


## FEATURE EXTRACTION WITH DATA AUGMENTATION and Fine-tuning
1 Add your custom network on top of an already-trained base network.

2 Freeze the base network.

3 Train the part you added.

4 Unfreeze some layers in the base network.

5 Jointly train both these layers and the part you added.

You already completed the first three steps when doing feature extraction. Let’s proceed with step 4: you’ll unfreeze your conv_base and then freeze individual layers inside it.

In [46]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [47]:
len(model.trainable_weights)

4

In [48]:
conv_base.trainable = True

In [49]:
len(model.trainable_weights)

30

#### Freezing all layers up to a specific one

In [50]:
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [51]:
len(model.trainable_weights)

10

#### Fine-tuning the model

In [52]:
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=1e-5),
metrics=['acc'])

In [53]:
epochs = 10 # 100
history = model.fit_generator(
                train_generator,
                steps_per_epoch=100,
                epochs=epochs,
                validation_data=validation_generator,
                validation_steps=50)

C:\Users\ESHOP\AppData\Local\Temp\ipykernel_6860\113632108.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 1.4916 - acc: 0.7380WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 50 batches). You may need to use the repeat() function when building your dataset.


100/100 [==============================] - 286s 3s/step - loss: 1.4916 - acc: 0.7380 - val_loss: 0.8812 - val_acc: 0.7500
Epoch 2/10
100/100 [==============================] - 279s 3s/step - loss: 0.3587 - acc: 0.8595
Epoch 3/10
100/100 [==============================] - 277s 3s/step - loss: 0.2707 - acc: 0.8933
Epoch 4/10
100/100 [==============================] - 278s 3s/step - loss: 0.2386 - acc: 0.9040
Epoch 5/10
100/100 [==============================] - 273s 3s/step - loss: 0.2098 - acc: 0.9260
Epoch 6/10
100/100 [==============================] - 279s 3s/step - loss: 0.1775 - acc: 0.9425
Epoch 7/10
100/100 [==============================] - 295s 3s/step - loss: 0.1898 - acc: 0.9385
Epoch 8/10
100/100 [==============================] - 303s 3s/step - loss: 0.1784 - acc: 0.9409
Epoch 9/10
100/100 [==============================] - 298s 3s/step - loss: 0.1625 - acc: 0.9405
Epoch 10/10
100/100 [==============================] - 324s 3s/step - loss: 0.1801 - acc: 0.9444


In [54]:
# saving model
model.save('model_F.h5')

## Evaluation

In [55]:
test_generator = test_datagen.flow_from_directory(
                    test_dir,
                    target_size=(150, 150),
                    batch_size=20,
                    class_mode='binary')

Found 624 images belonging to 2 classes.


In [56]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

C:\Users\ESHOP\AppData\Local\Temp\ipykernel_6860\1629466864.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)


test acc: 0.8717948794364929


In [ ]:
###################

## Saving it also in pickle format

In [57]:
import pickle

# Saving knn_user_model in binary mode 
Pickle = open('model', 'wb')
pickle.dump(model, Pickle)  
# close the file
Pickle.close()
#################
# Saving knn_model_user_movie in binary mode 
Pickle = open('model', 'wb')
pickle.dump(model, Pickle)  
# close the file
Pickle.close()
#################